# Coffea-Casa Processor-Based Workflow Test

This notebook demonstrates the UnifiedProcessor workflow with coffea.processor.Runner on Coffea-Casa, including skimming, analysis, histogramming, and statistics steps.

## Workflow Overview

1. Setup Python path for intccms package
2. Install dependencies and register modules for cloud pickle
3. Acquire Dask client from Coffea-Casa environment
4. Configure analysis parameters
5. Run metadata extraction
6. Initialize UnifiedProcessor
7. Run processor with coffea.processor.Runner
8. Save histograms
9. Run statistical analysis (if enabled)

## AF flag
We might want to run this code on different facilities, which may each have their own limitations or require different dask client setups. To make it easy to switch between facilities, just set the `AF` variable to the one of your choice. If your `AF` does not exist yet, you can introduce it in this notebook in the relevant sections.

In [1]:
AF="coffeacasa-condor" # options currently supported: [coffeacasa-condor, coffeacasa-gateway]

## Imports and dependencies

### The intccms package
The CMS implementation of the integration challenge is set in a package-like structure, which means we hae to add the source code to the python path. The package is referred to as `intccms`.

In [2]:
# Setup Python path to include intccms package
import sys
from pathlib import Path

# Add src directory to Python path
repo_root = Path.cwd()
src_dir = repo_root / "src"
examples_dir = repo_root
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))
if str(examples_dir) not in sys.path:
    sys.path.insert(0, str(examples_dir))
print(f"✅ Added {src_dir} to Python path")
print(f"✅ Added {examples_dir} to Python path")

✅ Added /home/cms-jovyan/intc/integration-challenge/cms/src to Python path
✅ Added /home/cms-jovyan/intc/integration-challenge/cms to Python path


### Installig extra dependencies
The `intccms` package requires `omegaconf`, which is not by default on an AF. 

In [3]:
try:
    import omegaconf
except ImportError:
    print("⚠️ omegaconf not found, installing...")
    ! pip install omegaconf

### Alternative coffea version
In some cases, we might need to install our own `coffea` version which is not on the AF. For example, when testing a new feature or using a recently realased version with a fix.

In [27]:
COFFEA_VERSION = "git+https://github.com/MoAly98/coffea.git@feat/fobj_in_procmeta"
COFFEA_PIP = f"coffea=={COFFEA_VERSION}" if "git" not in COFFEA_VERSION else COFFEA_VERSION

if AF == "coffeacasa-gateway" and "git" in COFFEA_VERSION:
    raise ValueError("The coffea-casa dask gateway facility does not support installing packages with pip via https on workers.")

# ! pip install $COFFEA_PIP

### Imports from stdlib and other libraries

In this notebook we use `dask` and `coffea`. 

In [5]:
# stdlib
import cloudpickle
from contextlib import contextmanager
import copy
import os
import time

from coffea.processor import DaskExecutor, IterativeExecutor
from coffea.nanoevents import NanoAODSchema
from dask.distributed import Client, PipInstall, WorkerPlugin

### Imports from intccms and other integration-challenge specific tooling

In [6]:
# intccms
from intccms.schema import Config, load_config_with_restricted_cli
from intccms.utils.output import OutputDirectoryManager
from intccms.metadata_extractor import DatasetMetadataManager
from intccms.datasets import DatasetManager
from intccms.analysis import run_processor_workflow

### Registering packages with cloudpickle
The intccms cannot be installed on the workers via `pip`, and the configuration files are in python modules which also cannot be installed on the workers. So we need to register them with `cloudpickle` to allow dask to serialize them and send them out.

In [7]:
import intccms
import example_cms

# Register modules for cloud pickle
cloudpickle.register_pickle_by_value(intccms)
cloudpickle.register_pickle_by_value(example_cms)

## Dask client setup

This notebook uses the `DaskExecutor` from `coffea` to distribute the task graph on the AF. The client setup varies in different facilities, so we implement a function which returns the correct client. The function does so by providing a context manager, within which the client is alive.

In [25]:
@contextmanager
def acquire_client(af):
    """Context manager to acquire and safely close a Dask client from a Coffea-Casa environment."""

    # These are the pip-installable dependncies which should be installed on workers
    dependencies = [COFFEA_PIP, "git+https://github.com/MoAly98/roastcoffea.git"]
    client, cluster = None, None
    
    try:
        # Coffea-casa condor facility
        if af == "coffeacasa-condor":
            client = Client("tls://localhost:8786")
            client.register_plugin(PipInstall(packages=dependencies))
            client.forward_logging()

        # Coffea-casa dask gateway facility
        elif af == "coffea-casa-gateway":
            from dask_gateway import Gateway
        
            def set_env(dask_worker):
                config_path = str(Path(dask_worker.local_directory) / 'access_token')
                os.environ["BEARER_TOKEN_FILE"] = config_path
                os.chmod(config_path, 0o600)
                os.chmod("/etc/grid-security/certificates", 0o755)

            num_workers = 150   #number of workers desired
            gateway = Gateway()
            clusters = gateway.list_clusters()
            cluster = gateway.connect(clusters[0].name)
            client = cluster.get_client()
            cluster.scale(num_workers)
            client.wait_for_workers(num_workers)
            client.upload_file("/etc/cmsaf-secrets-chown/access_token")
            client.register_worker_callbacks(setup=set_env)
            client.register_plugin(PipInstall(packages=dependencies))

        print(f"✅ Connected to Dask scheduler")
        print(f"📊 Dashboard: {client.dashboard_link}")

        yield client, cluster
    
    except Exception as e:
        raise e
        print(f"ERROR:: {e}")
    
    finally:
        if client is not None:
            client.close()
            print("✅ Client closed")

## Configuration Setup

The CMS analysis implementation is configurable via python modules, which we have to import. For this notebook, the configuration files are found in `example_cms/configs/`. You can modify the modules in this directory manually, or you can dynamically change settings using python dictionary manipulation. Below are some settings of interest that you might want to tune when you are testing your setup.

In [16]:
# intccms configuration import
from example_cms.configs.configuration import config as original_config

# Create a deepcopy that we can manipulate
config = copy.deepcopy(original_config)

# Limit files for testing
config["datasets"]["max_files"] = None # None would run over all availale files

# Use local output directory
config["general"]["output_dir"] = "example_cms/outputs/"

# Preprocessing (coffea) can be executed once and results loaded
config["general"]["run_metadata_generation"] = True # If True, run analysis pre-processing

# Processer = Skimming (filter and save) + Analysis
config["general"]["run_processor"] = True  # If True, the coffea processor is executed
config["general"]["run_analysis"] = True # If True, the analysis part of the processor is executed
config["general"]["save_skimmed_output"] = False  # If True, skimmed events are saved to disk, otherwise filter executed on-the-fly

# Analysis = Systematics + histogramming + statsitics
config["general"]["run_histogramming"] = True
config["general"]["run_systematics"] = True
config["general"]["run_statistics"] = False

# Datasets to process, by default this is all datasets
#config["general"]["processes"] = ["data"] 

cli_args = [] # the code can be ran from CLI, but we don't care here
full_config = load_config_with_restricted_cli(config, cli_args)

# Validated config gives us a dictionary object with all settings checked to be safe with pydantic
validated_config = Config(**full_config)

## Running the Workflow

Running the CMS integration challenge workflow is split into a few steps, with a modular design that allows us flexibility. The steps are:

1. Setting up output directories
2. Building an input dataset manager
3. Running or loading the coffea preprocessing
4. Run the coffea processor

### Output manager setup

In [17]:
output_manager = OutputDirectoryManager(
    root_output_dir=validated_config.general.output_dir,
    cache_dir=validated_config.general.cache_dir,
    metadata_dir=validated_config.general.metadata_dir,
    skimmed_dir=validated_config.general.skimmed_dir
)

23:29:16 INFO     Output directory manager initialized with root:                                ]8;id=570430;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/utils/output/directories.py\directories.py]8;;\:]8;id=527105;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/utils/output/directories.py#169\169]8;;\
                  /home/cms-jovyan/intc/integration-challenge/cms/example_cms/outputs                              

## Configure Data Redirector (Optional)

Override the redirector in the config for accessing dataset files. Useful for testing different storage backends. You can also change this in `example_cms/configs/skim.py`

In [18]:
# Override redirector for all datasets
# Examples:
#   "root://xcache/"                    
#   "root://cmsxrootd.fnal.gov/"
#   "root://cms-xrd-global.cern.ch/"
REDIRECTOR = "root://xcache/"  # Change this to use a different redirector

print(f"Initial redirector  {validated_config.datasets.datasets[0].name}: {validated_config.datasets.datasets[0].redirector}")

# Apply to all datasets in config
for dataset in validated_config.datasets.datasets:
 dataset.redirector = REDIRECTOR

print(f"Redirector set to: {REDIRECTOR}")

# Verify the change
print(f"New redirector:  {validated_config.datasets.datasets[0].name}: {validated_config.datasets.datasets[0].redirector}")

Initial redirector  signal: root://xcache/
Redirector set to: root://xcache/
New redirector:  signal: root://xcache/


### Input dataset manager setup

In [19]:
dataset_manager = DatasetManager(validated_config.datasets)

23:29:18 INFO     Initialized dataset manager with 10 datasets                                        ]8;id=644409;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/datasets/manager.py\manager.py]8;;\:]8;id=23260;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/datasets/manager.py#34\34]8;;\

### Coffea preprocessing

In [22]:
metadata_generator = DatasetMetadataManager(
  dataset_manager=dataset_manager,
  output_manager=output_manager,
  config=validated_config,
)

if metadata_generator.generate_metadata:
  with acquire_client(AF) as (client, cluster):
      metadata_generator.run(executor=DaskExecutor(client=client))
else:
  metadata_generator.run()  # No client needed

# Build metadata lookup and extract workitems
metadata_lookup = metadata_generator.build_metadata_lookup()
workitems = metadata_generator.workitems

23:31:31 INFO     Initialized DatasetMetadataManager with output dir:                                ]8;id=106734;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/metadata_extractor/manager.py\manager.py]8;;\:]8;id=706755;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/metadata_extractor/manager.py#130\130]8;;\
                  /home/cms-jovyan/intc/integration-challenge/cms/example_cms/outputs/metadata                     

✅ Connected to Dask scheduler
📊 Dashboard: /user/mohamed.aly@cern.ch/proxy/8787/status
<Client: 'tls://192.168.161.135:8786' processes=300 threads=300, memory=866.13 GiB>


23:31:35 INFO     Starting metadata generation workflow...                                           ]8;id=900220;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/metadata_extractor/manager.py\manager.py]8;;\:]8;id=578534;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/metadata_extractor/manager.py#205\205]8;;\

         INFO     Building fileset for process: signal                                              ]8;id=478306;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/metadata_extractor/builders.py\builders.py]8;;\:]8;id=633995;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/metadata_extractor/builders.py#123\123]8;;\

         INFO     Building fileset for process: ttbar_semilep                                       ]8;id=889964;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/metadata_extractor/builders.py\builders.py]8;;\:]8;id=729574;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/metadata_extractor/builders.py#123\123]8;;\

...

         INFO     Saved JSON to                                                                           ]8;id=536284;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/metadata_extractor/io.py\io.py]8;;\:]8;id=98517;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/metadata_extractor/io.py#107\107]8;;\
                  /home/cms-jovyan/intc/integration-challenge/cms/example_cms/outputs/metadata/nanoaods_d          
                  ata_12_nominal.json                                                                              

         INFO     Saved JSON to                                                                           ]8;id=775240;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/metadata_extractor/io.py\io.py]8;;\:]8;id=39992;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/metadata_extractor/io.py#107\107]8;;\
                  /home/cms-jovyan/intc/integration-challenge/cms/example_cms/outputs/metadata/nanoaods_d          
                  ata_13_nominal.json                                                                              

         INFO     Metadata generation complete.                                                      ]8;id=998345;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/metadata_extractor/manager.py\manager.py]8;;\:]8;id=870141;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/metadata_extractor/manager.py#226\226]8;;\

✅ Client closed


         INFO     Built metadata lookup for 125 fileset keys                                         ]8;id=350816;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/metadata_extractor/manager.py\manager.py]8;;\:]8;id=120960;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/metadata_extractor/manager.py#443\443]8;;\

### Run the coffea processor!

In [28]:
# Run processor workflow
from intccms.analysis.processor import UnifiedProcessor

with acquire_client(AF) as (client, cluster):
    # Create processor instance for MetricsCollector
    unified_processor = UnifiedProcessor(
        config=validated_config,
        output_manager=output_manager,
        metadata_lookup=metadata_lookup,
    )

    t0 = time.perf_counter()
    output, report = run_processor_workflow(
        config=validated_config,
        output_manager=output_manager,
        metadata_lookup=metadata_lookup,
        workitems=workitems,
        executor=DaskExecutor(client=client, treereduction=8, retries=0),
        schema=NanoAODSchema,
    )
    t1 = time.perf_counter()


print(f"✅ Processor workflow complete in {t1-t0:.1f} seconds!")

# Print summary
print(f"Total events processed: {output.get('processed_events', 0):,}")
print(f"Events after skim: {output.get('skimmed_events', 0):,}")

✅ Connected to Dask scheduler

📊 Dashboard: /user/mohamed.aly@cern.ch/proxy/8787/status

23:42:07 INFO     Initialized UnifiedProcessor: save_skimmed_output=False, analysis=True,          ]8;id=706227;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/analysis/processor.py\processor.py]8;;\:]8;id=806147;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/analysis/processor.py#195\195]8;;\
                  histogramming=True, systematics=True                                                             

         INFO     Running processor over data...                                                      ]8;id=73218;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/analysis/runner.py\runner.py]8;;\:]8;id=290801;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/analysis/runner.py#109\109]8;;\

         INFO     Initialized UnifiedProcessor: save_skimmed_output=False, analysis=True,          ]8;id=434162;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/analysis/processor.py\processor.py]8;;\:]8;id=137296;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/analysis/processor.py#195\195]8;;\
                  histogramming=True, systematics=True                                                             

...

         INFO     Processor complete: 6,827,277,103 events processed, 297,458,382 events after skim   ]8;id=854576;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/analysis/runner.py\runner.py]8;;\:]8;id=361274;file:///home/cms-jovyan/intc/integration-challenge/cms/src/intccms/analysis/runner.py#205\205]8;;\

✅ Client closed

✅ Processor workflow complete in 1047.1 seconds!

Total events processed: 6,827,277,103

Events after skim: 297,458,382